In [1]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# from IPython.display import display, HTML

# init_notebook_mode(connected=True)

In [22]:
import plotly
plotly.__version__

'2.5.1'

In [3]:
# from plotly.offline import init_notebook_mode, iplot
# from IPython.display import display, HTML

# import pandas as pd

# init_notebook_mode(connected=True)

In [30]:
footprint = pd.read_csv('../data/bubble chart/footprint.csv',
                       low_memory=False)

In [31]:
footprint.head(10)

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,1.400203e-01,0.199159,0.097000,0.036817,0.029258,0.000000e+00,5.022540e-01,5
1,Armenia,1992,1,AreaTotHA,4.830000e+05,687000.000000,334600.000000,127000.000000,100925.003052,0.000000e+00,1.732525e+06,5
2,Armenia,1992,1,BiocapPerCap,2.765314e-01,0.134892,0.083839,0.013705,0.057782,0.000000e+00,5.667493e-01,5
3,Armenia,1992,1,BiocapTotGHA,9.538950e+05,465308.532841,289203.573356,47274.017744,199320.619674,0.000000e+00,1.955002e+06,5
4,Armenia,1992,1,EFConsPerCap,4.774125e-01,0.175880,0.000001,0.004113,0.057782,1.097617e+00,1.812806e+00,5
5,Armenia,1992,1,EFConsTotGHA,1.646834e+06,606697.374570,4.328034,14188.301981,199320.619674,3.786229e+06,6.253274e+06,5
6,Armenia,1992,1,EFExportsPerCap,1.535785e-03,0.002071,0.000000,0.000449,0.000000,4.744830e-02,5.150366e-02,5
7,Armenia,1992,1,EFExportsTotGHA,5.297689e+03,7143.838664,0.000000,1547.439795,0.000000,1.636729e+05,1.776619e+05,5
8,Armenia,1992,1,EFImportsPerCap,2.024169e-01,0.056342,0.000001,0.003299,0.000000,8.822890e-02,3.502881e-01,5
9,Armenia,1992,1,EFImportsTotGHA,6.982370e+05,194350.774605,4.328034,11381.057213,0.000000,3.043456e+05,1.208319e+06,5


In [32]:
EFcons = footprint.loc[(footprint['record'] == 'EFConsPerCap'),
                       ['country', 'year', 'total']].reset_index(drop = True)   
EFcons.columns = [u'country', u'year', u'EFCons_total']

In [33]:
Biocap = footprint.loc[(footprint['record'] == 'BiocapPerCap'),
                       ['country', 'year', 'total']].reset_index(drop = True)   
Biocap.columns = [u'country', u'year', u'Biocap_total']

In [34]:
world_data = EFcons.merge(Biocap, on=['country','year'],how = 'left')
world_data['Delta'] = world_data['Biocap_total'] - world_data['EFCons_total'] 

In [35]:
world_data.head()

,country,year,EFCons_total,Biocap_total,Delta
0,Armenia,1992,1.812806,0.566749,-1.246057
1,Armenia,1993,1.215480,0.567299,-0.648181
2,Armenia,1994,1.156674,0.569013,-0.587662
3,Armenia,1995,1.061664,0.589461,-0.472203
4,Armenia,1996,1.069321,0.630939,-0.438382


### add country code

In [36]:
country = pd.read_csv('../data/bubble chart/country_code.csv')

In [37]:
country.loc[country['Country Code']=='USA', 'TableName'] = 'United States of America'

In [38]:
country.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba,NaN
1,AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan,NaN
2,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,ALB,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra,NaN


In [39]:
country = country[['Country Code','Region', 'TableName']]
country.columns = ['Country Code','Region', 'country']

In [40]:
world_data_country = world_data.merge(country, on='country',how = 'inner')

In [41]:
world_data_country['status'] = 'Reserve'
world_data_country.loc[world_data_country['Delta']<0, 'status' ] = 'Deficit'

In [42]:
world_data_country.head()

,country,year,EFCons_total,Biocap_total,Delta,Country Code,Region,status
0,Armenia,1992,1.812806,0.566749,-1.246057,ARM,Europe & Central Asia,Deficit
1,Armenia,1993,1.215480,0.567299,-0.648181,ARM,Europe & Central Asia,Deficit
2,Armenia,1994,1.156674,0.569013,-0.587662,ARM,Europe & Central Asia,Deficit
3,Armenia,1995,1.061664,0.589461,-0.472203,ARM,Europe & Central Asia,Deficit
4,Armenia,1996,1.069321,0.630939,-0.438382,ARM,Europe & Central Asia,Deficit


### one year plot - 2013

In [43]:
df = world_data_country[world_data_country['year'] == 2013]
df=df.reset_index(drop = True)
df.head()

,country,year,EFCons_total,Biocap_total,Delta,Country Code,Region,status
0,Armenia,2013,2.069790,0.893841,-1.175949,ARM,Europe & Central Asia,Deficit
1,Afghanistan,2013,0.888804,0.477357,-0.411447,AFG,South Asia,Deficit
2,Albania,2013,2.294674,1.229782,-1.064892,ALB,Europe & Central Asia,Deficit
3,Algeria,2013,2.381949,0.590312,-1.791638,DZA,Middle East & North Africa,Deficit
4,Angola,2013,0.979065,2.420093,1.441028,AGO,Sub-Saharan Africa,Reserve


In [44]:
df.describe()

,year,EFCons_total,Biocap_total,Delta
count,164.0,164.000000,164.000000,164.000000
mean,2013.0,3.263772,3.659213,0.395441
std,0.0,2.284430,9.300430,9.254501
min,2013.0,0.504504,0.052923,-11.507298
25%,2013.0,1.431032,0.701739,-2.053251
50%,2013.0,2.798333,1.513530,-0.701401
75%,2013.0,4.530455,2.888072,0.312591
max,2013.0,13.091594,89.435431,85.421248


In [45]:
df.loc[df['Delta']>15]

,country,year,EFCons_total,Biocap_total,Delta,Country Code,Region,status
43,Guyana,2013,3.316525,69.500349,66.183825,GUY,Latin America & Caribbean,Reserve
150,Gabon,2013,2.447503,26.194587,23.747084,GAB,Sub-Saharan Africa,Reserve
157,Suriname,2013,4.014183,89.435431,85.421248,SUR,Latin America & Caribbean,Reserve


In [46]:
df = df.loc[df['Delta']<15]

In [48]:
data = [ dict(
        type = 'choropleth',
        locations = df['Country Code'],
        z = df['Delta'],
        text = 'Footprint: ' + df['EFCons_total'].astype(str) + "<br>" \
                'Biocapacity: ' + df['Biocap_total'].astype(str) + "<br>" \
                + df['country'],
#         colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
#             [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = True,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
#             tickprefix = '$',
            title = 'Reserve/Deficit',
        )
        
      ) ]

layout = dict(
    title = '2013 Global EF Vs. Biocapacity',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    ),
)

fig = dict( data=data, layout=layout )
# iplot( fig, validate=False, filename='d3-world-map' )
plotly.offline.plot(fig, validate=False, filename='world-map2013.html') 

'file:///Users/jingsong/MSAN/06Spring2018B/2DataVisualization/project/visualization-Ecological-Footprint/code/world-map2013.html'

In [55]:
years = world_data_country['year'].sort_values().unique()
years

array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013])

In [56]:
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

In [57]:
# fill in most of layout
# figure['layout']['xaxis'] = {'range': [-30, 60], 'title': 'Ecological Footprint Growth %'}
# figure['layout']['yaxis'] = {'title': 'GDP Growth %', 'range': [-30, 60]}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1961',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}


figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
#         'x': 0.1,
#         'xanchor': 'right',
#         'y': 0,
#         'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [58]:
# make data
year = 1961

df = world_data_country[world_data_country['year'] == year]
df=df.reset_index(drop = True)
df = df.loc[df['Delta']<15]
data_dict = [ dict(
        type = 'choropleth',
        locations = df['Country Code'],
        z = df['Delta'],
        text = 'Footprint: ' + df['EFCons_total'].astype(str) + "<br>" \
                'Biocapacity: ' + df['Biocap_total'].astype(str) + "<br>" \
                + df['country'],
        autocolorscale = True,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
#             tickprefix = '$',
            title = 'Reserve/Deficit',
        )     
      ) ]
figure['data'].append(data_dict)

In [59]:
# make frames
for year in years:
    frame = {'data': [], 'name': str(year), 'layout': []}
    df = world_data_country[world_data_country['year'] == year]
    df = df.reset_index(drop = True)
    df = df.loc[df['Delta']<15]
    data_dict = [ dict(
        type = 'choropleth',
        locations = df['Country Code'],
        z = df['Delta'],
        text = 'Footprint: ' + df['EFCons_total'].astype(str) + "<br>" \
                'Biocapacity: ' + df['Biocap_total'].astype(str) + "<br>" \
                + df['country'],
#         colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
#             [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = True,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
#             tickprefix = '$',
            title = 'Reserve/Deficit',
        )     
      ) ]
    frame['data'] = data_dict
    
    layout = dict(
    title = 'Global Ecological Footprint and Biocapacity',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    ),
)
    frame['layout'] = layout
    
    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

# iplot(figure, validate=False,filename='d3-world-map-animated')
plotly.offline.plot(figure, validate=False, filename='world-map-all.html') 

'file:///Users/jingsong/MSAN/06Spring2018B/2DataVisualization/project/visualization-Ecological-Footprint/code/world-map-all.html'